c.f. `_bin_psd`

In [ ]:
import numpy as np

from tail.numba_wrap import fftfreq
from tail.pseudospectra import _bin_psd_hit_only

In [ ]:
from numba import jit

In [ ]:
import pandas as pd

In [ ]:
import plotly.express as px

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
# high-l v1
# width of square mask (with padding)
N = 2880
# from args-common.yml
# 2 arcminute in radian
pixel_size = np.pi / 5400.
l_max = 4901

In [ ]:
freq = fftfreq(N, pixel_size)

In [ ]:
hit = _bin_psd_hit_only(pixel_size, l_max, N)

In [ ]:
l = np.arange(l_max)

In [ ]:
# no. of modes in curve space
hit_predicted = 2 * l + 1

In [ ]:
# scaled by the ratio of the no. of mode between curve and flat spaces
ratio = hit.sum() / hit_predicted.sum()
hit_predicted = ratio * hit_predicted

In [ ]:
df = pd.DataFrame(
    np.column_stack((hit, hit_predicted)),
    index=pd.Index(l, name='l'),
    columns=pd.Index(['hit', 'hit-predicted'], name='type')
)

In [ ]:
df.plot(labels={'value': "hit"})

Noise

In [ ]:
from tail.analysis.container import PseudoSpectra, get_idx

In [ ]:
pseudo_spectra = PseudoSpectra.load_h5('/scratch/largepatch_new/high_1_0_2.hdf5')

In [ ]:
pseudo_spectra.names

In [ ]:
pseudo_spectra.realmaps.shape

In [ ]:
hit.shape

In [ ]:
# BB noise spectrum
noise = pseudo_spectra.realmaps[get_idx(pseudo_spectra.spectra, 'BB'), 0, 0, :, 1, 0]

In [ ]:
scale = l * (l + 1)
df_noise = pd.DataFrame(
    np.column_stack((noise / scale, noise * hit / scale)),
    index=pd.Index(l, name='l'),
    columns=pd.Index(('noise', 'noise times hit'), name='type')
)

In [ ]:
df_noise.loc[600:3000].plot(labels={'value': "noise"})

In [ ]:
signal = pseudo_spectra.signal

In [ ]:
BB = signal[get_idx(pseudo_spectra.spectra, 'BB'), 0, 0]

In [ ]:
BB

In [ ]:
df_BB = pd.DataFrame(
    np.column_stack((BB, BB * hit)),
    index=pd.Index(l, name='l'),
    columns=pd.Index(('BB', 'BB times hit'), name='type')
)

In [ ]:
df_BB.loc[600:3000].plot(labels={'value': "noise"})

In [ ]:
err = pseudo_spectra.err_mc

In [ ]:
BB_err = err[get_idx(pseudo_spectra.spectra, 'BB'), 0, 0]

In [ ]:
df_BB_err = pd.DataFrame(
    np.column_stack((BB_err, BB_err * hit)),
    index=pd.Index(l, name='l'),
    columns=pd.Index(('BB error-bar', 'BB error-bar times hit'), name='type')
)

In [ ]:
df_BB_err.loc[600:3000].plot(labels={'value': "noise"})